# Lab 10: Recommender System

In this assignment, we will study how to do user-based collaborative filtering and item-based collaborative filtering. 

## 1. Dataset

In this assignment, we will use MovieLens-100K dataset. It includes about 100,000 ratings from 1000 users on 1700 movies.  

In [3]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors


# 1. load data
user_ratings_train = pd.read_csv('./ml-100k/u1.base',
                            sep='\t',names=['user_id','movie_id','rating'], usecols=[0,1,2])

user_ratings_test = pd.read_csv('./ml-100k/u1.test',
                            sep='\t',names=['user_id','movie_id','rating'], usecols=[0,1,2])

movie_info =  pd.read_csv('./ml-100k/u.item', 
                          sep='|', names=['movie_id','title'], usecols=[0,1],
                          encoding="ISO-8859-1")

user_ratings_train = pd.merge(movie_info, user_ratings_train)
user_ratings_test = pd.merge(movie_info, user_ratings_test)

# 2. get the rating matrix. Each row is a user, and each column is a movie.
user_ratings_train = user_ratings_train.pivot_table(index=['user_id'],
                                        columns=['title'],
                                        values='rating')

user_ratings_test = user_ratings_test.pivot_table(index=['user_id'],
                                        columns=['title'],
                                        values='rating')




user_ratings_train = user_ratings_train.reindex(
                            index=user_ratings_train.index.union(user_ratings_test.index), 
                            columns=user_ratings_train.columns.union(user_ratings_test.columns) )

user_ratings_test = user_ratings_test.reindex(
                            index=user_ratings_train.index.union(user_ratings_test.index), 
                            columns=user_ratings_train.columns.union(user_ratings_test.columns) )

print(user_ratings_train.shape)
print(user_ratings_test.shape)

(943, 1664)
(943, 1664)


## Task 1. User-based CF

* Use pearson correlation to get the similarity between different users.
* Based on the obtained similarity score, predict the ratings. You can use 5 nearest neighbors or 10 nearest neighbors.
* Compute MAE for the testing set.

In [30]:
# your code
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_absolute_error

#fills Nan with average
user_ratings_train['avg'] = user_ratings_train.mean(axis=1)
user_ratings_train_noNan = user_ratings_train.T.fillna(user_ratings_train['avg'], axis=0).T

#pearson correlation
pearson_sim_train = 1-pairwise_distances(user_ratings_train_noNan, metric="correlation")
print(pearson_sim_train)
print("\n")

#5 nearest neighbors 
train_model = NearestNeighbors(n_neighbors=10)
train_model.fit(pearson_sim_train)
neighbors_distance, neighbors_ind = train_model.kneighbors()

#reseting index
neighbors_ind += 1

predictions = []
actual = []

#predict the ratings
for user_id, row in user_ratings_test.iterrows():
    for movie, rating in row.iteritems():
        if not pd.isnull(rating):
            predicted_rating = 0
            sum_of_sim = 0
            for x in range(0,10):
                ngbh_id = neighbors_ind[user_id-1][x]
                nghb_rating = user_ratings_train.loc[ngbh_id,movie]
                if not pd.isnull(nghb_rating):
                    nghb_distance = neighbors_distance[user_id-1][x]
                    sum_of_sim += nghb_distance
                    predicted_rating += nghb_distance*(nghb_rating-user_ratings_train.loc[ngbh_id, 'avg'])
            if (sum_of_sim != 0):
                predicted_rating = predicted_rating/sum_of_sim
                predicted_rating += user_ratings_train.loc[user_id, 'avg']
                predictions.append(predicted_rating)
                actual.append(rating)

#10 KNN has smaller MAE
mae = mean_absolute_error(predictions, actual)
print("MAE for testing set: " + str(mae))

[[ 1.00000000e+00  2.32208120e-02  1.32786679e-02 ...  6.16507063e-02
  -3.06075887e-02  5.98564252e-02]
 [ 2.32208120e-02  1.00000000e+00  9.51733735e-03 ...  5.34428840e-03
   2.50789352e-02  4.37874936e-02]
 [ 1.32786679e-02  9.51733735e-03  1.00000000e+00 ...  3.58078305e-02
   3.83776757e-02  5.34075010e-03]
 ...
 [ 6.16507063e-02  5.34428840e-03  3.58078305e-02 ...  1.00000000e+00
  -1.85501504e-02  6.66444773e-04]
 [-3.06075887e-02  2.50789352e-02  3.83776757e-02 ... -1.85501504e-02
   1.00000000e+00  4.11337777e-02]
 [ 5.98564252e-02  4.37874936e-02  5.34075010e-03 ...  6.66444773e-04
   4.11337777e-02  1.00000000e+00]]


MAE for testing set: 0.8316716024705649


## Task 2. Item-based CF
* Use cosine similarity to get the similarity between different items.
* Based on the obtained similarity score, predict the ratings. You can use 5 nearest neighbors or 10 nearest neighbors.
* Compute MAE for the testing set.

In [41]:
# your code
item_ratings_train = user_ratings_train.T
item_ratings_test = user_ratings_test.T

#fill Nans with row avg
item_ratings_train['avg'] = item_ratings_train.mean(axis=1)
item_ratings_train_noNan = item_ratings_train.T.fillna(item_ratings_train['avg'], axis=0).T

#drop rows with missing data  
dropped_rows = item_ratings_train_noNan[item_ratings_train_noNan.isna().any(axis=1)]
item_ratings_train_noNan = item_ratings_train_noNan.drop(dropped_rows.index)
item_ratings_test = item_ratings_test.drop(dropped_rows.index)

#cosine similarity
cosine_sim_train = 1-pairwise_distances(item_ratings_train_noNan, metric="cosine")
print(pearson_sim_train)
print("\n")

#5 nearest neighbors 
train_model = NearestNeighbors(n_neighbors=10)
train_model.fit(cosine_sim_train)
neighbors_distance, neighbors_ind = train_model.kneighbors()

#reseting index
neighbors_ind += 1

predictions = []
actual = []

#predicting ratings
for movie_id, row in item_ratings_test.iterrows():
    item_id = item_ratings_test.index.get_loc(movie_id)
    for user, rating in row.iteritems():
        if not pd.isnull(rating):
            predicted_rating = 0
            sum_of_sim = 0
            for x in range(0,10):
                ngbh_id = neighbors_ind[item_id][x]
                nghb_rating = item_ratings_train.iloc[ngbh_id].loc[user]
                if not pd.isnull(nghb_rating):
                    nghb_distance = neighbors_distance[item_id][x]
                    sum_of_sim += nghb_distance
                    predicted_rating += nghb_distance*(nghb_rating)
            if (sum_of_sim != 0):
                predicted_rating = predicted_rating/sum_of_sim
                predictions.append(predicted_rating)
                actual.append(rating)

#10 KNN has smaller MAE
mae = mean_absolute_error(predictions, actual)
print("MAE for testing set: " + str(mae))

[[1.         0.99940449 0.99317483 ... 0.99942103 0.99975181 0.99215062]
 [0.99940449 1.         0.99316242 ... 0.9992226  0.99965259 0.99230413]
 [0.99317483 0.99316242 1.         ... 0.99316242 0.99350757 0.98762425]
 ...
 [0.99942103 0.9992226  0.99316242 ... 1.         0.99965259 0.99196888]
 [0.99975181 0.99965259 0.99350757 ... 0.99965259 1.         0.99222378]
 [0.99215062 0.99230413 0.98762425 ... 0.99196888 0.99222378 1.        ]]


MAE for testing set: 1.0371091280012175
